<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/vaccine/%E5%89%AF%E5%8F%8D%E5%BF%9C%E7%96%91%E3%81%84%E5%A0%B1%E5%91%8A%E9%9B%86%E8%A8%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.mhlw.go.jp/content/10906000/000784435.pdf -O data.pdf

--2021-05-26 14:13:39--  https://www.mhlw.go.jp/content/10906000/000784435.pdf
Resolving www.mhlw.go.jp (www.mhlw.go.jp)... 173.223.150.129, 2600:1408:5400:48d::3194, 2600:1408:5400:4a8::3194
Connecting to www.mhlw.go.jp (www.mhlw.go.jp)|173.223.150.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4932198 (4.7M) [application/pdf]
Saving to: ‘data.pdf’

data.pdf            100%[===================>]   4.70M  --.-KB/s    in 0.1s    

2021-05-26 14:13:39 (47.8 MB/s) - ‘data.pdf’ saved [4932198/4932198]



In [2]:
!apt install python3-tk ghostscript
!pip install camelot-py[cv]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts libcupsfilters1
  libcupsimage2 libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 11 newly installed, 0 to remove and 34 not upgraded.
Need to get 14.1 MB of archives.
After this operation, 

In [3]:
import camelot
import pandas as pd

In [4]:
tables = camelot.read_pdf("data.pdf", pages="2-end", split_text=True)

df = pd.concat([table.df for table in tables])

df.to_csv("data.csv", encoding="utf_8_sig", index=False, header=False)

# ダウンロード

In [ ]:
from google.colab import files

In [ ]:
files.download("data.csv")

# 前処理

In [5]:
df0 = pd.read_csv("data.csv", dtype="object").fillna("")

In [6]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7298 entries, 0 to 7297
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   No             7298 non-null   object
 1   年齢             7298 non-null   object
 2   性別             7298 non-null   object
 3   接種日            7298 non-null   object
 4   発生日            7298 non-null   object
 5   接種から
発生までの
日数  7298 non-null   object
 6   ワクチン名          7298 non-null   object
 7   製造販売業者         7298 non-null   object
 8   ロット番
号         7298 non-null   object
 9   症状名（PT名）       7298 non-null   object
 10  因果関係
（報告医評価）   7298 non-null   object
 11  重篤度
（報告医評価）    7298 non-null   object
 12  転帰日            7298 non-null   object
 13  転帰内容           7298 non-null   object
dtypes: object(14)
memory usage: 798.3+ KB


In [7]:
df0.columns = df0.columns.str.replace("\n", "")

In [8]:
df0["No"] = df0["No"].str.replace("※[１-９]", "").str.strip().astype(int)

In [9]:
df0.set_index("No", inplace=True)

In [10]:
df0["年齢"] = df0["年齢"].str.rstrip("歳")
df0["年齢"] = df0["年齢"].mask(df0["年齢"] == "").astype(float).astype("Int64")

In [11]:
df0["性別"] = df0["性別"].replace({"": "不明"})

In [12]:
df1 = df0.copy()

# 症状別報告件数

In [13]:
def symptoms(df1):

    df2 = (
        df1["症状名（PT名）"]
        .str.replace("\s", "")
        .str.extractall("(.+?)（(.+?)）")
        .rename(columns={0: "症状名", 1: "PT名"})
        .reset_index()
        .rename(columns={"level_0": "No"})
    )

    df3 = df2.loc[:, ["No", "症状名"]].join(df2["PT名"].str.split("|", expand=True))

    df4 = (
        pd.melt(df3, id_vars=["No", "症状名"])
        .dropna(subset=["value"])
        .drop("variable", axis=1)
        .rename(columns={"value": "PT名"})
        .reset_index(drop=True)
    )

    df4["症状"] = df4["症状名"] + "（" + df4["PT名"] + "）"

    df4["症状"].value_counts().to_csv("症状別報告件数.csv", encoding="utf_8_sig")

symptoms(df1)

# アナフィラキシー

In [14]:
df1["症状名（PT名）"].str.contains("アナフィラキシー").sum()

994

# 性別

In [15]:
# 性別 副反応疑い報告数
df1["性別"].value_counts()

女性    6048
男性    1241
不明       9
Name: 性別, dtype: int64

In [16]:
# 性別 重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "性別"].value_counts()

女性    705
男性    139
不明      2
Name: 性別, dtype: int64

In [17]:
# 性別 死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "性別"].value_counts()

女性    29
男性    23
Name: 性別, dtype: int64

# 年齢別

In [18]:
labels = [
    "0～9歳",
    "10～19歳",
    "20～29歳",
    "30～39歳",
    "40～49歳",
    "50～59歳",
    "60～69歳",
    "70～79歳",
    "80歳以上",
]

In [19]:
df1["年代"] = pd.cut(
    df1["年齢"],
    [0, 10, 20, 30, 40, 50, 60, 70, 80, 120],
    include_lowest=True,
    right=False,
    labels=labels,
).astype(str)

In [20]:
df1["年代"] = df1["年代"].mask(df1["年齢"].isnull(), "不明")

In [21]:
# 年齢別副反応疑い報告数
df1["年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳         0
10～19歳      41
20～29歳    1491
30～39歳    1724
40～49歳    2063
50～59歳    1363
60～69歳     403
70～79歳      90
80歳以上      121
不明           2
Name: 年代, dtype: int64

In [22]:
# 年齢別重篤報告数
df1.loc[df1["重篤度（報告医評価）"] == "重い", "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳        0
10～19歳      5
20～29歳    149
30～39歳    197
40～49歳    246
50～59歳    152
60～69歳     38
70～79歳     20
80歳以上      37
不明          2
Name: 年代, dtype: int64

In [23]:
# 年齢別死亡報告数
df1.loc[df1["転帰内容"].str.contains("死亡"), "年代"].value_counts().reindex(labels + ["不明"]).fillna(0).astype(int)

0～9歳       0
10～19歳     0
20～29歳     3
30～39歳     1
40～49歳     5
50～59歳     4
60～69歳     6
70～79歳    10
80歳以上     23
不明         0
Name: 年代, dtype: int64